In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import topic_weights as tw
import os
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2

# Load Data

In [3]:
subdir = 'final_csvs2'

In [4]:
dataall = pd.DataFrame.from_csv(os.path.join(subdir,'dataall.csv'),encoding='utf-8')
datanonother = pd.DataFrame.from_csv(os.path.join(subdir,'datanonother.csv'),encoding='utf-8')

In [5]:
n_topics = 50
stem = 'stem'
package = 'sklearn'
rows = 'all'
twcsv = tw.topic_weights_csv(n_topics,stem,package,rows)
topicweights = pd.DataFrame.from_csv(os.path.join(subdir,twcsv))

# T-SNE

In [6]:
X_tsne = TSNE(learning_rate=100,verbose=1).fit_transform(topicweights)

[t-SNE] Computing pairwise distances...
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Computed conditional probabilities for sample 1000 / 9662
[t-SNE] Computed conditional probabilities for sample 2000 / 9662
[t-SNE] Computed conditional probabilities for sample 3000 / 9662
[t-SNE] Computed conditional probabilities for sample 4000 / 9662
[t-SNE] Computed conditional probabilities for sample 5000 / 9662
[t-SNE] Computed conditional probabilities for sample 6000 / 9662
[t-SNE] Computed conditional probabilities for sample 7000 / 9662
[t-SNE] Computed conditional probabilities for sample 8000 / 9662
[t-SNE] Computed conditional probabilities for sample 9000 / 9662
[t-SNE] Computed conditional probabilities for sample 9662 / 9662
[t-SNE] Mean sigma: 0.107976
[t-SNE] Error after 50 iterations with early exaggeration: 1.553268
[t-SNE] Error after 75 iterations: 1.553248


# Topic IDs

In [7]:
from collections import Counter
import operator as op
from model_predict import LDA, TF

In [8]:
tf_feature_names = TF.get_feature_names()

In [9]:
topicids = np.argmax(topicweights.as_matrix(),axis=1)

In [10]:
count = Counter(topicids)
sortedcounts = sorted(count.items(),key=op.itemgetter(1),reverse=True)

In [11]:
for sortedcount in sortedcounts[:15]:
    idx = sortedcount[0]
    print(idx, tw.top_words(LDA,tf_feature_names,n_top_words=8,topic_idx=idx))

(1, u'get say like one peopl dont go said')
(3, u'one live like peopl life world year us')
(33, u'polic offic prison crime crimin sentenc year arrest')
(12, u'state militari unit war forc govern turkey secur')
(18, u'health drug patient medic doctor diseas hospit cancer')
(47, u'govern polit power peopl democraci protest countri corrupt')
(21, u'bank financi chang economi polici would econom govern')
(11, u'parti elect polit minist leader nation govern vote')
(4, u'peopl like studi one experi research differ work')
(43, u'republican democrat voter polit candid parti elect campaign')
(5, u'court law rule state judg case feder legal')
(42, u'percent rate econom american incom growth year social')
(22, u'citi new york state mayor de public plan')
(41, u'tax money would million spend fund billion budget')
(10, u'bill state would senat law congress republican legisl')


In [12]:
besttopics = {33: 'Police and crime',
              12: 'Foreign policy',
              18: 'Health care',
              47: 'Government',
              21: 'Banks and finance',
              11: 'Elections',
              43: 'Domestic politics',
              5: 'Courts and law',
              42: 'Economy',
              22: 'New York'}

In [13]:
topicidsnew = [besttopics.get(topicid,'None') for topicid in topicids]

In [14]:
df = pd.DataFrame(index=topicweights.index,
                  columns=['Component 1','Component 2','Topic ID','Author','URL'])

In [15]:
df['Component 1'] = X_tsne[:,0]
df['Component 2'] = X_tsne[:,1]
df['Topic ID'] = topicidsnew
df['Author'] = dataall['author']
df['Title'] = dataall['title']
df['URL'] = dataall['url']
df['Share Count'] = dataall['share_count']

In [54]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib qt

In [16]:
dffinal = df[df['Topic ID'] != 'None']

In [75]:
topics = np.unique(dffinal['Topic ID'])
topicmapdict = {topic: i for i, topic in enumerate(topics)}
topicidid = [topicmapdict[topic] for topic in dffinal['Topic ID']]
dffinal['Topic ID ID'] = topicidid

/home/varun/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [80]:
dffinal.to_csv('topics.tsv',encoding='utf-8',index=False,sep='\t')

In [129]:
keys1 = ['Component 1','Component 2','Share Count','Topic ID','Topic ID ID','Author','Title']
keys2 = [u'one',u'two',u'shares',u'topic',u'tid',u'author',u'title']

In [130]:
with open('out','wb') as f:
    for row in dffinal.iterrows():
        mystring = u'{0},\n'.format(gen_row(row[1],keys1,keys2))
        f.write(mystring.encode('utf-8'))

In [51]:
dffinal.iloc[:20].to_json()

'{"Component 1":{"6":0.0009163953,"7":-0.0010415819,"9":0.0018586484,"11":-0.0004961732,"18":0.0030207311,"20":-0.0007793708,"26":0.0004521777,"28":0.0001189789,"31":0.0005486608,"33":-0.0000274088,"34":-0.0019830916,"35":0.0005942927,"36":0.0024595922,"41":-0.0012566696,"44":-0.0005113076,"60":-0.0044596354,"61":-0.0003398164,"63":0.0008182635,"64":-0.0000656738,"70":-0.0005676238},"Component 2":{"6":-0.0027222379,"7":0.0002454044,"9":-0.0005975703,"11":-0.0025408805,"18":-0.0007277086,"20":-0.0030368689,"26":0.0020754001,"28":0.0005640023,"31":0.0002900714,"33":0.0006554431,"34":-0.0004821063,"35":0.0008315095,"36":0.0022000438,"41":-0.0016885483,"44":-0.0024066371,"60":0.0003040575,"61":0.0003762935,"63":0.0009943908,"64":-0.0021624251,"70":0.0006865256},"Topic ID":{"6":"New York","7":"Police and crime","9":"Government","11":"Domestic politics","18":"Courts and law","20":"Foreign policy","26":"Domestic politics","28":"Government","31":"Police and crime","33":"Domestic politics","34"